# Debug for card art construction

card size are 63 x 88 mm (816 x 1110) (2.48 x 3.46 inches)

In [1]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageChops
import numpy as np
import polars as pl
import os
import random
import matplotlib.pyplot as plt
import sys
HOME_DIR = r'c:\Users\jordy\Documents\python\projects\GenAI_TCG'
sys.path.append(HOME_DIR)

In [2]:
def round_corners(im, radius):
    mask = Image.new('L', im.size, 0)
    draw_mask = ImageDraw.Draw(mask)
    draw_mask.rounded_rectangle([0, 0, im.width, im.height], radius=radius, fill=255)
    im_rounded = im.copy()
    im_rounded.putalpha(mask)
    return im_rounded

def rotate_image(im, angle, expand=True):
    return im.rotate(angle, expand=expand)

def white_to_transparent(im):
    im_np = np.array(im)
    white_threshold = 200
    white_mask = np.all(im_np[:, :, :3] > white_threshold, axis=2)
    im_np[white_mask, 3] = 0
    return Image.fromarray(im_np)

def im_resize_scale(im, scale_factor):
    new_width = int(im.width * scale_factor)
    new_height = int(im.height * scale_factor)
    new_size = (new_width, new_height)
    return im.resize(new_size, Image.LANCZOS)

def add_image_overlay(base_image, overlay_image_path, position_pct, round_corners_radius=None, white_to_transp=True, resize_scale=None, rotate=0):
    """
    position_pct: (x_pct, y_pct) where each is in [0,1] relative to base_image size
    resize_scale: scale factor for overlay image (relative to its original size)
    """
    overlay_image = Image.open(overlay_image_path).convert('RGBA')
    if white_to_transp:
        overlay_image = white_to_transparent(overlay_image)
    if resize_scale:
        overlay_image = im_resize_scale(overlay_image, resize_scale)
    if rotate:
        overlay_image = rotate_image(overlay_image, rotate)
    if round_corners_radius:
        overlay_image = round_corners(overlay_image, radius=round_corners_radius)
    x = int(base_image.width * position_pct[0])
    y = int(base_image.height * position_pct[1])
    base_image.paste(overlay_image, (x, y), overlay_image)
    return base_image

def add_text_overlay(base_image, text, position_pct, color, font_size_pct=0.1, rotate=0):
    font_size = int(base_image.width * font_size_pct)
    font = ImageFont.truetype(r'..\cards_assets\Aladin-Regular.ttf', size=font_size)
    x = int(base_image.width * position_pct[0])
    y = int(base_image.height * position_pct[1])
    # Measure text size and offset
    dummy_img = Image.new('RGBA', (1, 1))
    dummy_draw = ImageDraw.Draw(dummy_img)
    text_bbox = dummy_draw.textbbox((0, 0), text, font=font, anchor="mm")
    text_w = text_bbox[2] - text_bbox[0]
    text_h = text_bbox[3] - text_bbox[1]
    # Add padding to avoid cutting
    pad_top = abs(text_bbox[1])
    pad_bottom = max(0, text_bbox[3])
    pad_left = abs(text_bbox[0])
    pad_right = max(0, text_bbox[2])
    padded_w = text_w + pad_left + pad_right
    padded_h = text_h + pad_top + pad_bottom
    text_img = Image.new('RGBA', (padded_w, padded_h), (0, 0, 0, 0))
    text_draw = ImageDraw.Draw(text_img)
    # Draw text centered in padded image
    text_draw.text(
        (padded_w // 2, padded_h // 2),
        text,
        font=font,
        fill=color,
        anchor="mm"
    )
    if rotate:
        rotated_text = text_img.rotate(rotate, expand=True)
        px = x - rotated_text.width // 2
        py = y - rotated_text.height // 2
        base_image.alpha_composite(rotated_text, dest=(px, py))
    else:
        base_image.alpha_composite(text_img, dest=(x - padded_w // 2, y - padded_h // 2))
    return base_image

def blur_region(im, x_pct, y_pct, w_pct, h_pct, gauss_radius=5, corner_radius_pct=0.1, transp_edge_percent=0.2):
    x = int(im.width * x_pct)
    y = int(im.height * y_pct)
    w = int(im.width * w_pct)
    h = int(im.height * h_pct)
    region = im.crop((x, y, x + w, y + h))
    blurred = region.filter(ImageFilter.GaussianBlur(radius=gauss_radius))
    mask = Image.new('L', (w, h), 0)
    mask_np = np.zeros((h, w), dtype=np.float32)
    edge_x = int(w * transp_edge_percent)
    edge_y = int(h * transp_edge_percent)
    for i in range(h):
        for j in range(w):
            dx = min(j, w - 1 - j)
            dy = min(i, h - 1 - i)
            fx = min(1.0, dx / edge_x) if edge_x > 0 else 1.0
            fy = min(1.0, dy / edge_y) if edge_y > 0 else 1.0
            mask_np[i, j] = fx * fy
    mask_np = (mask_np * 255).astype(np.uint8)
    mask = Image.fromarray(mask_np, mode='L')
    if corner_radius_pct > 0:
        corner_mask = Image.new('L', (w, h), 0)
        draw_mask = ImageDraw.Draw(corner_mask)
        draw_mask.rounded_rectangle([0, 0, w, h], radius=int(im.width * corner_radius_pct), fill=255)
        mask = ImageChops.multiply(mask, corner_mask)
    blended = Image.composite(blurred, region, mask)
    im.paste(blended, (x, y))
    return im

def color_region(im, x_pct, y_pct, w_pct, h_pct, color="#FFFFFF", corner_radius_pct=10, transp_edge_percent=0.2):
    x = int(im.width * x_pct)
    y = int(im.height * y_pct)
    w = int(im.width * w_pct)
    h = int(im.height * h_pct)
    overlay = Image.new('RGBA', (w, h), color)
    mask_np = np.zeros((h, w), dtype=np.float32)
    edge_x = int(w * transp_edge_percent)
    edge_y = int(h * transp_edge_percent)
    for i in range(h):
        for j in range(w):
            dx = min(j, w - 1 - j)
            dy = min(i, h - 1 - i)
            fx = min(1.0, dx / edge_x) if edge_x > 0 else 1.0
            fy = min(1.0, dy / edge_y) if edge_y > 0 else 1.0
            mask_np[i, j] = fx * fy
    mask_np = (mask_np * 255).astype(np.uint8)
    mask = Image.fromarray(mask_np, mode='L')
    if corner_radius_pct > 0:
        corner_mask = Image.new('L', (w, h), 0)
        draw_mask = ImageDraw.Draw(corner_mask)
        draw_mask.rounded_rectangle([0, 0, w, h], radius=int(im.width * corner_radius_pct), fill=255)
        mask = ImageChops.multiply(mask, corner_mask)
    region = im.crop((x, y, x + w, y + h))
    blended = Image.composite(overlay, region, mask)
    im.paste(blended, (x, y))
    return im

def transparent_colored_overlay(im, x_pct, y_pct, w_pct, h_pct, transparency_percent=20, color=(255, 255, 255), corner_radius_pct=0):
    """
    Adds a semi-transparent, colored overlay of a specified size to an image, with optional rounded corners.

    Args:
        im: PIL.Image.Image, input image
        x_pct, y_pct, w_pct, h_pct: region as percent of image size
        transparency_percent: The transparency percentage (0-100). 0=opaque, 100=fully transparent.
        color: The RGB or RGBA tuple for the color of the overlay.
        corner_radius_pct: percent of image width for rounded corners.
    """
    x = int(im.width * x_pct)
    y = int(im.height * y_pct)
    w = int(im.width * w_pct)
    h = int(im.height * h_pct)

    # Ensure the color tuple is RGB
    if len(color) == 4:
        color = color[:3]
    if len(color) != 3:
        print("Error: The 'color' argument must be an RGB tuple (e.g., (255, 0, 0) for red).")
        return

    if not 0 <= transparency_percent <= 100:
        print("Error: Transparency percentage must be between 0 and 100.")
        return

    opacity = 1 - (transparency_percent / 100)
    alpha_value = int(opacity * 255)
    overlay_color = color + (alpha_value,)

    # Create the smaller transparent overlay
    transparent_layer = Image.new('RGBA', (w, h), overlay_color)

    # Create rounded corners mask if needed
    if corner_radius_pct > 0:
        corner_mask = Image.new('L', (w, h), 0)
        draw_mask = ImageDraw.Draw(corner_mask)
        radius = int(im.width * corner_radius_pct)
        draw_mask.rounded_rectangle([0, 0, w, h], radius=radius, fill=255)
        # Apply mask to overlay alpha channel
        overlay_np = np.array(transparent_layer)
        mask_np = np.array(corner_mask)
        overlay_np[..., 3] = (overlay_np[..., 3] * (mask_np / 255)).astype(np.uint8)
        transparent_layer = Image.fromarray(overlay_np, mode='RGBA')

    final_image = im.copy()
    final_image.paste(transparent_layer, (x, y), transparent_layer)

    return final_image


## Test on one image at a time

In [ ]:
imBase_path = r'..\cards\Dwa23_7fe05a.png' # dwarves

blurring_radius = 8

imBase = Image.open(imBase_path).convert('RGBA')

# 0 - colored patches on the base image before blurring it (helping see numbers or markers/icons)
# imBase = color_region(imBase, 1, 60, 45, 45, color="#DFDFDF", corner_radius=10)
# imBase = color_triangle(imBase, -210, 400, int(200*1.8), int(120*1.8), color="#000000")

# region 1 - blur first so that the layers comes on top
imBase = transparent_colored_overlay(imBase, 0.01, 0.01, 0.14, 0.25, transparency_percent=40, color=(0, 0, 0), corner_radius_pct=0.02) # Dark transparent overlay for Condition / Effect
imBase = blur_region(imBase, 0.01, 0.01, 0.14, 0.25, gauss_radius=blurring_radius, corner_radius_pct=0.02, transp_edge_percent=0.0)   # (w start %, h start %, w %, h %) Blurring for top-left icons

imBase = transparent_colored_overlay(imBase, 0.05, 1-0.20-0.005, 0.90, 0.20, transparency_percent=40, color=(0, 0, 0), corner_radius_pct=0.05) # Dark transparent overlay for Condition / Effect
imBase = blur_region(imBase, 0.05, 1-0.20-0.005, 0.90, 0.20, gauss_radius=blurring_radius, corner_radius_pct=0.05, transp_edge_percent=0.0)   # (w start %, h start %, w %, h %) Blurring for Condition / Effect

imBase = transparent_colored_overlay(imBase, 0.5-(0.15/2), 1-0.20-0.01-0.1, 0.15, 0.1, transparency_percent=40, color=(0, 0, 0), corner_radius_pct=0.03) # Dark transparent overlay for Condition / Effect
imBase = blur_region(imBase, 0.5-(0.15/2), 1-0.20-0.01-0.1, 0.15, 0.1, gauss_radius=blurring_radius, corner_radius_pct=0.03, transp_edge_percent=0.0)  # (w start %, h start %, w %, h %) Faction logo
# endregion

# 2 - markers & texts
# region  2.1 - Top left markers
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_mana.png', (0.005, 0.01), round_corners_radius=None, white_to_transp=False, resize_scale=0.6)                 # Mana marker
imBase = add_text_overlay(imBase, str(3), (0.005+0.075, 0.077), color="#FFFFFF", font_size_pct=0.07)                                                            # Mana cost

imBase = add_text_overlay(imBase, str(2), (0.005+0.072+0.003, 0.145+0.003), color="#ffcb7d", font_size_pct=0.08)                                                # Value number shadow
imBase = add_text_overlay(imBase, str(2), (0.005+0.072, 0.145), color="#3B0C67", font_size_pct=0.08)                                                            # Value number

imBase = add_image_overlay(imBase, r'..\cards_assets\marker_shield.png', (0.005+0.015, 0.168), round_corners_radius=None, white_to_transp=False, resize_scale=0.5, rotate=90)         # shield marker
imBase = add_text_overlay(imBase, str(4), (0.082, 0.216), color='white', font_size_pct=0.06, rotate=90)                                                                           # shield value
# endregion

#   2.2 - Faction logo
imBase = add_image_overlay(imBase, r'..\cards_assets\logo_Dwarves.png', (0.425, 0.685), round_corners_radius=None, white_to_transp=False, resize_scale=0.2)                 # Faction marker
    

#   2.3 - condition & effect
rescale = 0.16
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_oppo_unique.png', (0.2, 0.77), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.35)
imBase = add_image_overlay(imBase, r'..\cards_assets\cond_face_left.png', (0.265, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)

imBase = color_region(imBase, 0.5-(0.005/2)-0.002, 0.81-0.005, 0.008, 0.13, color="#C5C5C5", corner_radius_pct=0.0, transp_edge_percent=0.15) # SEPARATOR back
imBase = color_region(imBase, 0.5-(0.005/2), 0.81, 0.005, 0.12, color="#171717", corner_radius_pct=0.0, transp_edge_percent=0.15) # SEPARATOR front

imBase = add_image_overlay(imBase, r'..\cards_assets\marker_player.png', (0.50, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.30)                   # player icon
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_mana.png', (0.57, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.52)                   # mana icon
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.68, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)      # arrow
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_bin.png', (0.755, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.19)                   # mana icon

# imBase = add_image_overlay(imBase, r'..\cards_assets\marker_instant.png', (0.83, 0.79), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.18)


#   2.4 - card name banner & card name
imBase = color_region(imBase, 0.08, 1-0.05-0.01, 1-0.08*2, 0.05, color="#2076C1", corner_radius_pct=0.03, transp_edge_percent=0.0)    # Black banner -border-  under card name
imBase = color_region(imBase, 0.1, 1-0.05-0.01, 0.8, 0.05, color="#000000", corner_radius_pct=0.03, transp_edge_percent=0.0)    # Black banner under card name
imBase = add_text_overlay(imBase, 'Brok Brouk', (0.5, 1-0.035), color="#7B422C", font_size_pct=0.05)   # card name

#   2.5 - biomes
imBase = add_image_overlay(imBase, r'..\cards_assets\biome_ocean.png', (0.1, 0.935), round_corners_radius=None, white_to_transp=False, resize_scale=0.1)              # 1st biome
imBase = add_image_overlay(imBase, r'..\cards_assets\biome_mountain.png', (0.815, 0.935), round_corners_radius=None, white_to_transp=False, resize_scale=0.1)              # 1st biome

imBase = imBase.resize((816, 1110), Image.LANCZOS)
imBase = round_corners(imBase, radius=30)   # Round the corners of the final card

imBase = imBase.resize((816, 1110), Image.LANCZOS)
imBase = round_corners(imBase, radius=15)   # Round the corners of the final card
imBase.save('im_layered_result.png')
imBase.show()

## Test on images in lib\artdesign\cards from card pool db

In [3]:
def add_layer_to_a_card(imBase_path, faction, mana_cost, advancing, shield, condition, effect, effect_number, rare, name, id):
    blurring_radius = 8
    if faction == "Dwarves":
        faction_logo_path = f'..\cards_assets\logo_{faction.lower()}.png'
        biome1_logo_path = r'..\cards_assets\biome_ocean.png'
        biome2_logo_path = r'..\cards_assets\biome_mountain.png'
        banner_border_color = "#236CA5"
        banner_txt_color = "#236CA5"
    elif faction == "Demons":
        faction_logo_path = f'..\cards_assets\logo_{faction.lower()}.png'
        biome1_logo_path = r'..\cards_assets\biome_ocean.png'
        biome2_logo_path = r'..\cards_assets\biome_desert.png'
        banner_border_color = "#3E1B6A"
        banner_txt_color = "#3E1B6A"
    elif faction == "Twigs":
        faction_logo_path = f'..\cards_assets\logo_{faction.lower()}.png'
        biome1_logo_path = r'..\cards_assets\biome_jungle.png'
        biome2_logo_path = r'..\cards_assets\biome_ocean.png'
        banner_border_color = "#4A7D3A"
        banner_txt_color = "#4A7D3A"
    elif faction == "Miaous":
        faction_logo_path = f'..\cards_assets\logo_{faction.lower()}.png'
        biome1_logo_path = r'..\cards_assets\biome_jungle.png'
        biome2_logo_path = r'..\cards_assets\biome_desert.png'
        banner_border_color = "#FF8253"
        banner_txt_color = "#FF8253"
    elif faction == "Orcs":
        faction_logo_path = f'..\cards_assets\logo_{faction.lower()}.png'
        biome1_logo_path = r'..\cards_assets\biome_mountain.png'
        biome2_logo_path = r'..\cards_assets\biome_jungle.png'
        banner_border_color = "#780B0B"
        banner_txt_color = "#780B0B"
    elif faction == "Mummies":
        faction_logo_path = f'..\cards_assets\logo_{faction.lower()}.png'
        biome1_logo_path = r'..\cards_assets\biome_desert.png'
        biome2_logo_path = r'..\cards_assets\biome_mountain.png'
        banner_border_color = "#FDF5E6"
        banner_txt_color = "#FDF5E6"

    font_color = "#FFFFFF"
    drop_shadow = "#ffcb7d"

    if rare:
        banner_border_color = "#FF3B93"
        banner_txt_color = "#FF3B93"
        font_color = "#FF3B93"
        drop_shadow = "#FF3B93"
    # print(condition, effect)
    imBase = Image.open(imBase_path).convert('RGBA')
    
    # region 1 - blur first so that the layers comes on top
    imBase = transparent_colored_overlay(imBase, 0.01, 0.01, 0.14, 0.25, transparency_percent=40, color=(0, 0, 0), corner_radius_pct=0.02) # Dark transparent overlay for Condition / Effect
    imBase = blur_region(imBase, 0.01, 0.01, 0.14, 0.25, gauss_radius=blurring_radius, corner_radius_pct=0.02, transp_edge_percent=0.0)   # (w start %, h start %, w %, h %) Blurring for top-left icons
    
    imBase = transparent_colored_overlay(imBase, 0.05, 1-0.20-0.005, 0.90, 0.20, transparency_percent=40, color=(0, 0, 0), corner_radius_pct=0.05) # Dark transparent overlay for Condition / Effect
    imBase = blur_region(imBase, 0.05, 1-0.20-0.005, 0.90, 0.20, gauss_radius=blurring_radius, corner_radius_pct=0.05, transp_edge_percent=0.0)   # (w start %, h start %, w %, h %) Blurring for Condition / Effect
    
    imBase = transparent_colored_overlay(imBase, 0.5-(0.15/2), 1-0.20-0.01-0.1, 0.15, 0.1, transparency_percent=40, color=(0, 0, 0), corner_radius_pct=0.03) # Dark transparent overlay for Condition / Effect
    imBase = blur_region(imBase, 0.5-(0.15/2), 1-0.20-0.01-0.1, 0.15, 0.1, gauss_radius=blurring_radius, corner_radius_pct=0.03, transp_edge_percent=0.0)  # (w start %, h start %, w %, h %) Faction logo
    # endregion

    # 2 - markers & texts
    # region  2.1 - Top left markers
    imBase = add_image_overlay(imBase, r'..\cards_assets\marker_mana.png', (0.005, 0.01), round_corners_radius=None, white_to_transp=False, resize_scale=0.6)                 # Mana marker
    imBase = add_text_overlay(imBase, str(mana_cost), (0.005+0.075, 0.077), color=font_color, font_size_pct=0.07)                                                            # Mana cost

    imBase = add_text_overlay(imBase, str(advancing), (0.005+0.072+0.003, 0.145+0.003), color=drop_shadow, font_size_pct=0.08)                                                # Value number shadow
    imBase = add_text_overlay(imBase, str(advancing), (0.005+0.072, 0.145), color="#3B0C67", font_size_pct=0.08)                                                            # Value number

    imBase = add_image_overlay(imBase, r'..\cards_assets\marker_shield.png', (0.005+0.015, 0.168), round_corners_radius=None, white_to_transp=False, resize_scale=0.5, rotate=90)         # shield marker
    imBase = add_text_overlay(imBase, str(shield), (0.082, 0.216), color=font_color, font_size_pct=0.06, rotate=90)                                                                           # shield value
    # endregion

    #   2.2 - Faction logo
    imBase = add_image_overlay(imBase, faction_logo_path, (0.425, 0.685), round_corners_radius=None, white_to_transp=False, resize_scale=0.2)                 # Faction marker
    

    #   region 2.3 - condition
    pos_x_delta = 0
    pos_y_delta = 0
    rescale = 0.18
    if condition == "mana_inf_6":
        condition_image_path = r'..\cards_assets\marker_mana.png'
        rescale = 0.18*3
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '< 6', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '< 6', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "mana_sup_5":
        condition_image_path = r'..\cards_assets\marker_mana.png'
        rescale = 0.18*3
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '> 5', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '> 5', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "cards_in_hand_inf_4":
        condition_image_path = r'..\cards_assets\effect_draw_p.png'
        pos_x_delta = 0.05
        rescale = 0.30
        imBase = add_text_overlay(imBase, '< 4', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '< 4', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "cards_in_hand_sup_3":
        condition_image_path = r'..\cards_assets\effect_draw_p.png'
        pos_x_delta = 0.05
        rescale = 0.30
        imBase = add_text_overlay(imBase, '> 3', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '> 3', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "dist_behind_sup_1":
        condition_image_path = r'..\cards_assets\cond_distance_behind.png'
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '> 1', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '> 1', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "dist_behind_sup_3":
        condition_image_path = r'..\cards_assets\cond_distance_behind.png'
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '> 3', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '> 3', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "dist_ahead_sup_1":
        condition_image_path = r'..\cards_assets\cond_distance_ahead.png'
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '> 1', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '> 1', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "dist_ahead_sup_3":
        condition_image_path = r'..\cards_assets\cond_distance_ahead.png'
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '> 3', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '> 3', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "pending":
        pos_x_delta = 0.05
        pos_y_delta = 0.035
        rescale = 0.20
        condition_image_path = r'..\cards_assets\marker_pending.png'
    elif condition == "drop_on_board":
        condition_image_path = r'..\cards_assets\marker_parachute.png'
    elif condition == "cataclysm":
        condition_image_path = r'..\cards_assets\cond_cata.png'
    elif condition == "day":
        condition_image_path = r'..\cards_assets\cond_day.png'
    elif condition == "night":
        condition_image_path = r'..\cards_assets\cond_night.png'
    elif condition == "temp_sup_9":
        condition_image_path = r'..\cards_assets\cond_hot_T.png'
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '> 9', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '> 9', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "temp_sup_15":
        condition_image_path = r'..\cards_assets\cond_hot_T.png'
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '> 15', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '> 15', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "temp_inf_11":
        condition_image_path = r'..\cards_assets\cond_cold_T.png'
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '< 11', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '< 11', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "temp_inf_6":
        condition_image_path = r'..\cards_assets\cond_cold_T.png'
        pos_x_delta = 0.05
        imBase = add_text_overlay(imBase, '< 6', (0.35+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '< 6', (0.35,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif condition == "face_point_right":
        condition_image_path = r'..\cards_assets\cond_face_right.png'
        pos_x_delta = -0.075
        pos_y_delta = -0.005
        rescale = 0.16
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_oppo_unique.png', (0.2, 0.77), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.35)
    elif condition == "face_point_left":
        condition_image_path = r'..\cards_assets\cond_face_left.png'
        pos_x_delta = -0.065
        pos_y_delta = -0.005
        rescale = 0.16
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_oppo_unique.png', (0.2, 0.77), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.35)

    if condition == 'no_condition':
        pass
    elif condition == "biome_Dwa":
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_mountain.png', (0.1, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
        imBase = add_text_overlay(imBase, '/', (0.28+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '/', (0.28,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_ocean.png', (0.32, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
    elif condition == "biome_Dem":
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_ocean.png', (0.1, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
        imBase = add_text_overlay(imBase, '/', (0.28+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '/', (0.28,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_desert.png', (0.32, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
    elif condition == "biome_Twi":
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_jungle.png', (0.1, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
        imBase = add_text_overlay(imBase, '/', (0.28+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '/', (0.28,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_ocean.png', (0.32, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
    elif condition == "biome_Mia":
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_jungle.png', (0.1, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
        imBase = add_text_overlay(imBase, '/', (0.28+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '/', (0.28,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_desert.png', (0.32, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
    elif condition == "biome_Orc":
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_mountain.png', (0.1, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
        imBase = add_text_overlay(imBase, '/', (0.28+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '/', (0.28,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_jungle.png', (0.32, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
    elif condition == "biome_Mum":
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_desert.png', (0.1, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
        imBase = add_text_overlay(imBase, '/', (0.28+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, '/', (0.28,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\biome_mountain.png', (0.32, 0.82), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
    else:
        imBase = add_image_overlay(imBase, condition_image_path, (0.2-pos_x_delta, 0.82-pos_y_delta), round_corners_radius=0.0, white_to_transp=False, resize_scale=rescale)
    # endregion 2.3 - condition

    imBase = color_region(imBase, 0.5-(0.005/2)-0.002, 0.81-0.005, 0.008, 0.13, color=drop_shadow, corner_radius_pct=0.0, transp_edge_percent=0.15) # SEPARATOR back
    imBase = color_region(imBase, 0.5-(0.005/2), 0.81, 0.005, 0.12, color="#25083F", corner_radius_pct=0.0, transp_edge_percent=0.15) # SEPARATOR Front

    #   region 2.4 - effect
    if effect == "advancing":
        effect_image_path = r'..\cards_assets\effect_advancing_p.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.55, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)                 # effect
        imBase = add_text_overlay(imBase, f'+{effect_number}', (0.75+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, f'+{effect_number}', (0.75,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif effect == 'draw':
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_p.png', (0.50, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)                   # card
        imBase = add_text_overlay(imBase, f'+{effect_number}', (0.585,       0.88), color='white', font_size_pct=0.06, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.665, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)                    # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\effect_draw_p.png', (0.75, 0.815), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.30)                   # draw icon
    elif effect == 'jump':
            effect_image_path = r'..\cards_assets\effect_jump.png'
            imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
    elif effect == 'wrecking_ball':
        effect_image_path = r'..\cards_assets\effect_wreckingball.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_instant.png', (0.83, 0.79), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.18)
    elif effect == 'advancing_oppo':
        effect_image_path = r'..\cards_assets\effect_advancing_oppo.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.55, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)                 # effect
        imBase = add_text_overlay(imBase, f'+{effect_number}', (0.75+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, f'+{effect_number}', (0.75,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif effect == 'backward':
        effect_image_path = r'..\cards_assets\effect_regressing_p.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.55, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)                 # effect
        imBase = add_text_overlay(imBase, f'{effect_number}', (0.75+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, f'{effect_number}', (0.75,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif effect == 'backward_oppo':
        effect_image_path = r'..\cards_assets\effect_regressing_oppo.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.55, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)                 # effect
        imBase = add_text_overlay(imBase, f'{effect_number}', (0.75+0.003, 0.88+0.003), color="#0d1936", font_size_pct=0.11)       # Value number shadow
        imBase = add_text_overlay(imBase, f'{effect_number}', (0.75,       0.88), color='white', font_size_pct=0.1, rotate=0)       # Value number
    elif effect == 'draw_oppo':
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_oppo.png', (0.50, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)                   # card
        imBase = add_text_overlay(imBase, f'+{effect_number}', (0.585,       0.88), color='white', font_size_pct=0.05, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.665, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)                    # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\effect_draw_oppo.png', (0.75, 0.815), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.30)                   # draw icon
    elif effect == 'discard':
        imBase = add_image_overlay(imBase, r'..\cards_assets\effect_draw_p.png', (0.51, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.30)                   # draw icon
        imBase = add_text_overlay(imBase, f'{effect_number}', (0.605,       0.85), color='white', font_size_pct=0.04, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.665, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)                    # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_bin.png', (0.75, 0.815), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.20)                    # bin
    elif effect == 'discard_oppo':
        imBase = add_image_overlay(imBase, r'..\cards_assets\effect_draw_oppo.png', (0.51, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.30)                   # draw icon
        imBase = add_text_overlay(imBase, f'{effect_number}', (0.605,       0.85), color='white', font_size_pct=0.04, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.665, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)                    # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_bin.png', (0.75, 0.815), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.20)                    # bin
    elif effect == 'unstoppable':
        effect_image_path = r'..\cards_assets\effect_unstoppable.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
    elif effect == 'ramp':  # add to mana
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_p.png', (0.50, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)                   # card
        imBase = add_text_overlay(imBase, f'+{1}', (0.585,       0.88), color='white', font_size_pct=0.06, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.66, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)                    # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_mana.png', (0.755, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.60)                   # mana icon
    elif effect == 'ramp_oppo':
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_oppo.png', (0.50, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)                   # card
        imBase = add_text_overlay(imBase, f'+{1}', (0.585,       0.88), color='white', font_size_pct=0.06, rotate=0)       # Value number
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.66, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)                    # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_mana.png', (0.755, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.60)                   # mana icon
    elif effect == 'taxation': # remove from mana
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_player.png', (0.50, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.30)                   # player icon
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_mana.png', (0.57, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.52)                   # mana icon
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.68, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)      # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_bin.png', (0.755, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.19)                   # mana icon
    elif effect == 'taxation_oppo':
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_opponent.png', (0.50, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.30)                   # player icon
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_mana.png', (0.57, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.52)                   # mana icon
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.68, 0.83), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)      # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_bin.png', (0.755, 0.825), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.19)                   # mana icon
    elif effect == 'grappling_hook':
        effect_image_path = r'..\cards_assets\effect_grapplinghook.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.18)
    # ---- faction dependent effects ----
    elif effect == 'pet_trap':      # Miaous
        effect_image_path = r'..\cards_assets\effect_pettrap.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_instant.png', (0.83, 0.79), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.18)
    elif effect == 'avalanche':     # Dwarves
        effect_image_path = r'..\cards_assets\cata_avalanche.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
    elif effect == 'rooted':         # Twigs
        effect_image_path = r'..\cards_assets\effect_rooted.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
    elif effect == 'copy_effect':   # Mummies
        effect_image_path = r'..\cards_assets\effect_copyeffect.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
    elif effect == 'effect_canceled': # Demons
        effect_image_path = r'..\cards_assets\effect_cancelspell.png'
        imBase = add_image_overlay(imBase, effect_image_path, (0.6, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
    elif effect == 'swap_cards':    # Orcs
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_unique.png', (0.48, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.62, 0.815), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=-20)   # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_arrow.png', (0.62, 0.855), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.12, rotate=180-20)   # arrow
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_card_unique.png', (0.69, 0.81), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.24)
        imBase = add_image_overlay(imBase, r'..\cards_assets\marker_instant.png', (0.83, 0.79), round_corners_radius=0.0, white_to_transp=False, resize_scale=0.18)
    # endregion 2.4 - effect


    #   2.4 - card name banner & card name
    imBase = color_region(imBase, 0.08, 1-0.05-0.01, 1-0.08*2, 0.05, color=banner_border_color, corner_radius_pct=0.03, transp_edge_percent=0.0)    # Black banner -border-  under card name
    imBase = color_region(imBase, 0.1, 1-0.05-0.01, 0.8, 0.05, color="#000000", corner_radius_pct=0.03, transp_edge_percent=0.0)    # Black banner under card name
    imBase = add_text_overlay(imBase, name, (0.5, 1-0.035), color=banner_txt_color, font_size_pct=0.05)   # card name

    #   2.5 - biomes
    imBase = add_image_overlay(imBase, biome1_logo_path, (0.1, 0.935), round_corners_radius=None, white_to_transp=False, resize_scale=0.1)              # 1st biome
    imBase = add_image_overlay(imBase, biome2_logo_path, (0.815, 0.935), round_corners_radius=None, white_to_transp=False, resize_scale=0.1)              # 1st biome

    imBase = imBase.resize((816, 1110), Image.LANCZOS)
    imBase = round_corners(imBase, radius=30)   # Round the corners of the final card
    
    output_filename = os.path.join(HOME_DIR, "lib", "artdesign", "cards_framed", f"{id}.png")
    imBase.save(output_filename)
    
    # imBase.show()
    return imBase

<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:10: SyntaxWarning: invalid escape sequence '\c'
<>:16: SyntaxWarning: invalid escape sequence '\c'
<>:22: SyntaxWarning: invalid escape sequence '\c'
<>:28: SyntaxWarning: invalid escape sequence '\c'
<>:34: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:10: SyntaxWarning: invalid escape sequence '\c'
<>:16: SyntaxWarning: invalid escape sequence '\c'
<>:22: SyntaxWarning: invalid escape sequence '\c'
<>:28: SyntaxWarning: invalid escape sequence '\c'
<>:34: SyntaxWarning: invalid escape sequence '\c'
C:\Users\jordy\AppData\Local\Temp\ipykernel_11972\1603462009.py:4: SyntaxWarning: invalid escape sequence '\c'
  faction_logo_path = f'..\cards_assets\logo_{faction.lower()}.png'
C:\Users\jordy\AppData\Local\Temp\ipykernel_11972\1603462009.py:10: SyntaxWarning: invalid escape sequence '\c'
  faction_logo_path = f'..\cards_assets\logo_{faction.lower()}.png'
C:\Users\jordy\AppData\Local\Te

In [4]:
# choose the faction !!!
fac_name = "Twigs"  # Dwarves, Demons, Twigs, Miaous Orcs, Mummies 
CARDPOOL = pl.read_parquet(os.path.join(HOME_DIR, "lib", "cardpool", f"fac_{fac_name}.parquet"))
CARDPOOL.head(5)

faction,mana,advancing,shield,condition,effect,effect_number,rare,condeff_value,card_id,prompt,negative_prompt,name
str,i64,i64,i64,str,str,i64,bool,i64,str,str,str,str
"""Twigs""",1,1,1,"""no_condition""","""advancing""",1,false,0,"""Twi11_2fafb1""","""An old small bark-covered male…","""""","""Zygote Pinecrush"""
"""Twigs""",1,0,1,"""no_condition""","""advancing""",2,false,-1,"""Twi11_c359fc""","""An old large bark-covered fema…","""""","""Hazel Mossbeard"""
"""Twigs""",1,-1,1,"""no_condition""","""advancing""",3,false,-2,"""Twi11_181ea0""","""A thin covered with moss male …","""""","""Dendron Leaf-armia"""
"""Twigs""",1,-1,1,"""no_condition""","""draw""",1,false,-2,"""Twi11_838008""","""A small covered with moss male…","""""","""Glimmerwood Elderwood"""
"""Twigs""",1,-2,1,"""no_condition""","""draw""",2,false,-3,"""Twi11_af7a8c""","""A young tall thorny male trean…","""""","""Kindling Sycamore"""


In [5]:
card_art_dir = os.path.join(HOME_DIR, "lib", "artdesign", "cards")
card_art_files = [f for f in os.listdir(card_art_dir) if os.path.isfile(os.path.join(card_art_dir, f))]
card_art_files = [f for f in card_art_files if fac_name[:3] in f]

debug = None # set to None to process all cards
if debug is not None:
    card_art_files = random.sample(card_art_files, debug)

imgs = []
for card_name in card_art_files:
    card_name_no_ext = os.path.splitext(card_name)[0]
    card_row = CARDPOOL.filter(pl.col("card_id") == card_name_no_ext)
    imBase_path = os.path.join(card_art_dir, card_name)
    try:
        imgs.append(add_layer_to_a_card(
            imBase_path=imBase_path,
            faction=card_row['faction'][0],
            mana_cost=card_row['mana'][0],
            advancing=card_row['advancing'][0],
            shield=card_row['shield'][0],
            condition=card_row['condition'][0],
            effect=card_row['effect'][0],
            effect_number=card_row['effect_number'][0],
            rare=card_row['rare'][0],
            name=card_row['name'][0],
            id=card_row['card_id'][0])
        )
    except Exception as e:
        print(f"Error processing card {card_name_no_ext}: {card_row['condition'][0]}, {card_row['effect'][0]}")

if debug is not None:
    ncols = min(4, len(imgs))
    nrows = (len(imgs) + ncols - 1) // ncols
    plt.figure(figsize=(6 * ncols, 8 * nrows), facecolor='black')
    for i, img in enumerate(imgs):
        plt.subplot(nrows, ncols, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.gca().set_facecolor('black')
    plt.tight_layout()
    plt.show()

C:\Users\jordy\AppData\Local\Temp\ipykernel_11972\404482701.py:182: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  transparent_layer = Image.fromarray(overlay_np, mode='RGBA')
C:\Users\jordy\AppData\Local\Temp\ipykernel_11972\404482701.py:100: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  mask = Image.fromarray(mask_np, mode='L')
C:\Users\jordy\AppData\Local\Temp\ipykernel_11972\404482701.py:127: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  mask = Image.fromarray(mask_np, mode='L')


In [ ]:
card_art_dir = os.path.join(HOME_DIR, "lib", "artdesign", "cards")
card_art_files = [f for f in os.listdir(card_art_dir) if os.path.isfile(os.path.join(card_art_dir, f))]
card_art_files = [f for f in card_art_files if fac_name[:3] in f]


## old

### first version not in % compared to im base (card) size

In [ ]:
def round_corners(im, radius):
    # Create a mask with rounded corners
    mask = Image.new('L', im.size, 0)
    draw_mask = ImageDraw.Draw(mask)
    draw_mask.rounded_rectangle([0, 0, im.width, im.height], radius=radius, fill=255)
    # Apply mask to image alpha channel
    im_rounded = im.copy()
    im_rounded.putalpha(mask)
    return im_rounded

def rotate_image(im, angle, expand=True):
    """
    Rotates a PIL image by the given angle.
    Args:
        im: PIL.Image.Image object.
        angle: Angle in degrees. Positive values rotate counter-clockwise.
        expand: Whether to expand the output image to hold the whole rotated image.
    Returns:
        Rotated PIL.Image.Image object.
    """
    return im.rotate(angle, expand=expand)

def white_to_transparent(im):
    # Convert overlay to numpy array
    im_np = np.array(im)

    # Define white threshold (tolerance for "almost white")
    white_threshold = 200

    # Create mask where all channels are above threshold (white)
    white_mask = np.all(im_np[:, :, :3] > white_threshold, axis=2)

    # Set alpha channel to 0 where mask is True (white)
    im_np[white_mask, 3] = 0

    # Convert back to PIL Image
    return Image.fromarray(im_np)

def im_resize_scale(im, scale_factor):
    # Calculate new size
    new_width = int(im.width * scale_factor)
    new_height = int(im.height * scale_factor)
    new_size = (new_width, new_height)
    return im.resize(new_size, Image.LANCZOS)

def add_image_overlay(base_image, overlay_image_path, position, round_corners_radius=None, white_to_transp=True, resize_scale=None, rotate=0):
    overlay_image = Image.open(overlay_image_path).convert('RGBA')
    if white_to_transp:
        overlay_image = white_to_transparent(overlay_image)
    if resize_scale:
        overlay_image = im_resize_scale(overlay_image, resize_scale)
    if rotate:
        overlay_image = rotate_image(overlay_image, rotate)
    if round_corners_radius:
        overlay_image = round_corners(overlay_image, radius=round_corners_radius)
    base_image.paste(overlay_image, position, overlay_image)
    return base_image

def add_text_overlay(base_image, text, position, color, font_size=40, rotate=0):
    font = ImageFont.truetype(r'..\cards_assets\Aladin-Regular.ttf', size=font_size)
    draw = ImageDraw.Draw(base_image)
    if rotate:
        # Create a transparent image for the text
        text_img = Image.new('RGBA', (font_size*2, font_size*2), (0, 0, 0, 0))
        text_draw = ImageDraw.Draw(text_img)
        text_draw.text((0, 0), text, font=font, fill=color)
        # Rotate the text image
        rotated_text = text_img.rotate(rotate, expand=False)
        # Composite the rotated text onto the base image
        base_image.alpha_composite(rotated_text, dest=position)
    else:
        draw.text(position, text, font=font, fill=color, anchor="mm")
    return base_image

# region old blur_region function
# def blur_region(im, x, y, w, h, radius=5, corner_radius=10):
#     # Crop the region to blur
#     region = im.crop((x, y, x + w, y + h))
#     # Apply Gaussian blur
#     blurred = region.filter(ImageFilter.GaussianBlur(radius=radius))

#     if corner_radius > 0:
#         # Create a mask with rounded corners
#         mask = Image.new('L', (w, h), 0)
#         draw_mask = ImageDraw.Draw(mask)
#         draw_mask.rounded_rectangle([0, 0, w, h], radius=corner_radius, fill=255)
#         # Paste using the mask for rounded corners
#         im.paste(blurred, (x, y), mask)
#     else:
#         # Paste the blurred region back (no rounded corners)
#         im.paste(blurred, (x, y))
#     return im
# endregion

def blur_region(im, x, y, w, h, radius=5, corner_radius=10, edge_percent=0.2):
    """
    Blurs a rectangular region of the image, with the blur effect strongest in the center and fading toward the edges.

    Args:
        im: PIL.Image.Image object.
        x, y: Top-left coordinates of the region.
        w, h: Width and height of the region.
        radius: Maximum blur radius.
        corner_radius: Corner radius for rounded corners.
        edge_percent: Fraction of width/height from the edge where blur starts to fade (0 to 0.5).
    """
    # Crop the region to blur
    region = im.crop((x, y, x + w, y + h))
    blurred = region.filter(ImageFilter.GaussianBlur(radius=radius))

    # Create a mask for blending: 1 in center, 0 at edges
    mask = Image.new('L', (w, h), 0)
    mask_np = np.zeros((h, w), dtype=np.float32)

    # Calculate edge fade
    edge_x = int(w * edge_percent)
    edge_y = int(h * edge_percent)

    for i in range(h):
        for j in range(w):
            dx = min(j, w - 1 - j)
            dy = min(i, h - 1 - i)
            fx = min(1.0, dx / edge_x) if edge_x > 0 else 1.0
            fy = min(1.0, dy / edge_y) if edge_y > 0 else 1.0
            mask_np[i, j] = fx * fy

    # Normalize and convert to 0-255
    mask_np = (mask_np * 255).astype(np.uint8)
    mask = Image.fromarray(mask_np, mode='L')

    if corner_radius > 0:
        # Apply rounded corners to the mask
        corner_mask = Image.new('L', (w, h), 0)
        draw_mask = ImageDraw.Draw(corner_mask)
        draw_mask.rounded_rectangle([0, 0, w, h], radius=corner_radius, fill=255)
        mask = ImageChops.multiply(mask, corner_mask)

    # Composite blurred and original region using the mask
    blended = Image.composite(blurred, region, mask)
    im.paste(blended, (x, y))

    return im

def color_region(im, x, y, w, h, color="#FFFFFF", corner_radius=10, edge_percent=0.2):
    """
    Overlays a colored rectangle (with optional rounded corners and edge fade) on a region of the image.

    Args:
        im: PIL.Image.Image object.
        x, y: Top-left coordinates of the region.
        w, h: Width and height of the region.
        color: Fill color in hex (e.g., "#FF0000").
        corner_radius: Corner radius for rounded corners.
        edge_percent: Fraction of width/height from the edge where color starts to fade (0 to 0.5).
    """
    # Create a color overlay
    overlay = Image.new('RGBA', (w, h), color)
    
    # Create a mask for blending: 1 in center, 0 at edges
    mask_np = np.zeros((h, w), dtype=np.float32)
    edge_x = int(w * edge_percent)
    edge_y = int(h * edge_percent)
    for i in range(h):
        for j in range(w):
            dx = min(j, w - 1 - j)
            dy = min(i, h - 1 - i)
            fx = min(1.0, dx / edge_x) if edge_x > 0 else 1.0
            fy = min(1.0, dy / edge_y) if edge_y > 0 else 1.0
            mask_np[i, j] = fx * fy
    mask_np = (mask_np * 255).astype(np.uint8)
    mask = Image.fromarray(mask_np, mode='L')

    if corner_radius > 0:
        # Apply rounded corners to the mask
        corner_mask = Image.new('L', (w, h), 0)
        draw_mask = ImageDraw.Draw(corner_mask)
        draw_mask.rounded_rectangle([0, 0, w, h], radius=corner_radius, fill=255)
        mask = ImageChops.multiply(mask, corner_mask)

    # Composite overlay and original region using the mask
    region = im.crop((x, y, x + w, y + h))
    blended = Image.composite(overlay, region, mask)
    im.paste(blended, (x, y))

    return im

def color_triangle(im, x, y, w, h, color="#FFFFFF"):
    """
    Overlays a colored triangle (with optional rounded corners and edge fade) on a region of the image.

    Args:
        im: PIL.Image.Image object.
        x, y: Top-left coordinates of the region.
        w, h: Width and height of the region.
        color: Fill color in hex (e.g., "#FF0000").
        corner_radius: Corner radius for rounded corners.
        edge_percent: Fraction of width/height from the edge where color starts to fade (0 to 0.5).
    """
    # Create a triangle overlay
    overlay = Image.new('RGBA', (w, h), (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)

    # Define triangle points
    top = (w // 2, 0)
    left = (0, h)
    right = (w, h)
    draw.polygon([top, left, right], fill=color)

    # Composite overlay and original region using the mask
    region = im.crop((x, y, x + w, y + h))
    blended = Image.alpha_composite(region, overlay)
    im.paste(blended, (x, y))

    return im

In [ ]:
# imBase_path = r'..\cards_ex\ComfyUI_00708_.png' # dwarf
# imBase_path = r'..\cards_ex\ComfyUI_00710_.png' # tuareg
imBase_path = r'..\cards_ex\ComfyUI_00773_.png' # bears

imBase = Image.open(imBase_path).convert('RGBA')

# -1 - add a bright square under the advancing number before blurring
imBase = color_region(imBase, 6, 60, 45, 45, color="#DFDFDF", corner_radius=10)
# imBase = color_triangle(imBase, -210, 400, int(200*1.8), int(120*1.8), color="#000000")

# 0 - blur first so that the layers comes on top
imBase = blur_region(imBase, -30, -30, 85, 200, radius=8, corner_radius=30, edge_percent=0.08)   # (w start, h start, w, h) Blurring for top-left icons
imBase = blur_region(imBase, 10, 420, 380, 100, radius=8, corner_radius=40, edge_percent=0.08)  # (w start, h start, w, h) Blurring 1 for conditions / effects
imBase = blur_region(imBase, int(400/2-(110/2)), 360, 110, 100, radius=8, corner_radius=40, edge_percent=0.08)  # (w start, h start, w, h) Blurring 2 for conditions / effects

# 1 - top-left markers
# mana droplet icon - (h, w) start pos
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_mana.png', (-5, 5), round_corners_radius=None, white_to_transp=True, resize_scale=0.3)
# mana text number
imBase = add_text_overlay(imBase, '3', (26, 42), color='white', font_size=25)
# advancing number white shadow
imBase = add_text_overlay(imBase, '+1', (23+2, 85+2), color="#F1C554", font_size=27)
# imBase = add_text_overlay(imBase, '+1', (16-2, 67+2), color="#F1C554", font_size=27)
# imBase = add_text_overlay(imBase, '+1', (16+2, 67+2), color="#F1C554", font_size=27)
# imBase = add_text_overlay(imBase, '+1', (16+2, 67-2), color="#F1C554", font_size=27)
# advancing value
imBase = add_text_overlay(imBase, '+1', (23, 85), color="#0A193A", font_size=27)
# SHIELD
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_shield.png', (3, 100), round_corners_radius=None, white_to_transp=True, resize_scale=0.22, rotate=90)
imBase = add_text_overlay(imBase, '5', (13, 90), color='white', font_size=19, rotate=90)

# 2 - Center Faction logo & biomes
imBase = add_image_overlay(imBase, r'..\cards_assets\logo_bears.png', (158, 355), round_corners_radius=None, white_to_transp=True, resize_scale=0.13)
imBase = add_image_overlay(imBase, r'..\cards_assets\logo_biome_mountain.png', (150, 415), round_corners_radius=None, white_to_transp=True, resize_scale=0.04, rotate=0)
imBase = add_image_overlay(imBase, r'..\cards_assets\logo_biome_jungle.png', (225, 415), round_corners_radius=None, white_to_transp=True, resize_scale=0.04, rotate=0)

# 3 - condition & effect
# imBase = add_image_overlay(imBase, r'..\cards_assets\marker_player.png', (400, 400), round_corners_radius=None, white_to_transp=True, resize_scale=0.13)  # 3.1 - cond
imBase = color_region(imBase, int(400/2-3), 445, 5, 50, color="#171717", corner_radius=10) # separate condition & effect
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_player.png', (205, 435), round_corners_radius=None, white_to_transp=True, resize_scale=0.13)  # 3.2 - effect
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_draw.png', (260, 435), round_corners_radius=None, white_to_transp=True, resize_scale=0.13)  # 3.2 - effect suite
imBase = add_text_overlay(imBase, '+1', (325+2, 435+30+2), color="#000000", font_size=35, rotate=0)
imBase = add_text_overlay(imBase, '+1', (325, 435+30), color="#FFFFFF", font_size=30)

# Banner - (h, w)
imBase = add_image_overlay(imBase, r'..\cards_assets\marker_banner.png', (-30, 490), round_corners_radius=None, white_to_transp=True, resize_scale=.385)
# Card name
imBase = add_text_overlay(imBase, 'Varryyyyy loooong naaaame', (200, 545), color="#8EA7D9", font_size=20)

imBase = round_corners(imBase, radius=15)   # Round the corners

imBase.save('im_layered_result.png')
imBase.show()